In [1]:
import pandas as pd
import warnings
import numpy as np
import pickle
import sys
import os

In [2]:
sys.path.append(os.path.abspath('C:/Users/teeyob/Predictive_Portfolio_Optimizer/scripts'))

In [3]:
from time_series_forecasting import train_arima, train_sarima, train_lstm, forecast_lstm, forecast_arima, evaluate_forecast
from data_analysis_functions import download_data

In [4]:
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-01-01'
end_date = '2024-10-31'

In [5]:
data = download_data(tickers, start_date, end_date)

[*********************100%***********************]  3 of 3 completed


Split Data into Training and Test Data

In [6]:
data.index = pd.to_datetime(data.index)  

train_data = data.loc[:'2022']

test_data = data.loc['2023':]

train_data_tsla = train_data['TSLA']

test_data_tsla = test_data['TSLA']



Train ARIMA model

In [7]:
warnings.filterwarnings("ignore", message="No supported index is available")
train_data_tsla.index = pd.to_datetime(train_data_tsla.index)
test_data_tsla.index = pd.to_datetime(test_data_tsla.index)
arima_model = train_arima(train_data_tsla, order=(1, 1, 1))

c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Forecast using the trained ARIMA model

In [8]:
arima_forecast = arima_model.get_forecast(steps=len(test_data_tsla)).predicted_mean
arima_forecast.index = test_data.index  

Evaluate the forecast

In [9]:
print("ARIMA Evaluation:", evaluate_forecast(test_data_tsla, arima_forecast))
print("First few rows of the forecast:")
print(arima_forecast.head())


ARIMA Evaluation: {'MAE': np.float64(2.634863224980734), 'RMSE': 3.2087385512118876, 'MAPE': np.float64(3.6819162312720297)}
First few rows of the forecast:
Date
2023-01-03 00:00:00+00:00    67.771154
2023-01-04 00:00:00+00:00    67.781589
2023-01-05 00:00:00+00:00    67.788503
2023-01-06 00:00:00+00:00    67.793084
2023-01-09 00:00:00+00:00    67.796119
Name: predicted_mean, dtype: float64


Train SARIMA Model

In [10]:
sarima_model = train_sarima(train_data_tsla, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_predictions = sarima_model.forecast(steps=len(test_data_tsla))


c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


SARIMA Evaluation

In [11]:
print("SARIMA Evaluation:", evaluate_forecast(test_data_tsla, sarima_predictions))

SARIMA Evaluation: {'MAE': np.float64(3.491453559272748), 'RMSE': 4.2568598812155765, 'MAPE': np.float64(4.876342057504622)}


LSTM Model

In [12]:
lstm_model, n_steps = train_lstm(train_data_tsla.values, n_steps=50)
lstm_predictions = forecast_lstm(lstm_model, test_data_tsla.values, n_steps=n_steps)
print("LSTM Evaluation:", evaluate_forecast(test_data_tsla.values[n_steps:], lstm_predictions))

Epoch 1/20


c:\Users\teeyob\Predictive_Portfolio_Optimizer\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 9006.8887
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 83.8047
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 24.0777
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 18.2421
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 14.0880
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 13.2236
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 11.0338
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 8.5044
Epoch 9/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 7.9227
Epoch 10/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 6.8136
Epoch 11/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 5.2748
Epoch 12/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 4.3946
Epoch 13/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 4.1812
Epoch 14/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.8446
Epoch 15/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.1389 
Epoch 

In [13]:
# Save ARIMA model
with open('arima_model.pkl', 'wb') as file:
    pickle.dump(arima_model, file)

# Save SARIMA model
with open('sarima_model.pkl', 'wb') as file:
    pickle.dump(sarima_model, file)

# Save LSTM model
with open('lstm_model.pkl', 'wb') as file:
    pickle.dump(lstm_model, file)
